In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Importation des données
df_population = pd.read_csv("data_sql/df_population.csv")
df_principale = pd.read_csv('data_sql/df_prinicpale.csv')
df_sous_alimentation = pd.read_csv('data_sql/df_sous_alimentation2013.csv')

# Prétraitement des données pour répecter le cahier de charge

In [3]:
# df_population
df_population['Valeur'] = df_population['Valeur']*1000
df_population = df_population[df_population['Année']==2013][['Zone','Code zone','Année','Valeur']]
df_population.rename(columns={'Zone':'pays','Code zone':'code_pays','Année':'annee','Valeur':'population'},inplace = True)
df_population.to_csv("data_sql/population.csv", index = False)

In [4]:
# df_dispo_alim
df_principale['annee'] = 2013
df_principale['dispo_alim_tonnes'] = \
df_principale['Disponibilité alimentaire en quantité (kg/personne/an)'] * df_principale['Population']/1000

df_dispo_alim = df_principale[['Zone', 'Code zone', 'annee', 'Produit','Type',
                           'dispo_alim_tonnes', 'Disponibilité alimentaire (Kcal/personne/jour)',
                           'Disponibilité de protéines en quantité (g/personne/jour)',
                           'Disponibilité de matière grasse en quantité (g/personne/jour)']].copy()

df_dispo_alim.rename(columns = {'Zone':'pays', 'Code zone':'code_pays', 'Produit':'produit',
                                'Type':'origin', 'Disponibilité alimentaire (Kcal/personne/jour)':'dispo_alim_kcal_p_j',
                                'Disponibilité de protéines en quantité (g/personne/jour)':'dispo_prot',
                                'Disponibilité de matière grasse en quantité (g/personne/jour)':'dispo_mat_gr'}, inplace=True)                

df_dispo_alim.to_csv('data_sql/dispo_alim.csv', index = False)

In [5]:
#df_equilibre_prod
df_equilibre_prod = df_principale[['Zone', 'Code zone', 'annee', 'Produit', 'Disponibilité intérieure (kg/an)',
                             'Aliments pour animaux (kg/an)','Semences (kg/an)','Pertes (kg/an)',
                              'Traitement (kg/an)', 'Nourriture (kg/an)','Autres utilisations (non alimentaire) (kg/an)'
                              ]].copy()  
df_equilibre_prod.rename(columns = {'Zone':'pays', 'Code zone':'code_pays', 'Produit':'produit',
                                 'Disponibilité intérieure (kg/an)':'dispo_int', 
                                 'Aliments pour animaux (kg/an)':'alim_ani','Semences (kg/an)':'semences',
                                 'Pertes (kg/an)':'pertes', 'Traitement (kg/an)':'transfo','Nourriture (kg/an)':'nourriture',
                                 'Autres utilisations (non alimentaire) (kg/an)':'autres_utilisations'},inplace=True)

df_equilibre_prod.to_csv('data_sql/equilibre_prod.csv', index = False)

In [6]:
# df_sous_nutrition
# df_sous_alimentation2013['Année']=2013
#df_sous_nutrition = df_sous_alimentation.groupby(['Zone','Code zone']).sum().reset_index().copy()
df_sous_nutrition = df_sous_alimentation[['Zone', 'Code zone', 'Année', 'Valeur']].copy()
df_sous_nutrition.rename(columns={'Zone':'pays', 'Code zone':'code_pays', 'Année':'annee',
                                         'Valeur':'nb_personnes'}, inplace=True)
df_sous_nutrition['nb_personnes'] *= 1000000
df_sous_nutrition.to_csv('data_sql/sous_nutrition.csv', index = False)

In [7]:
# Récupérer le contenu des tables
# df_population = pd.read_csv("data_sql/population.csv")
# df_dispo_alim = pd.read_csv('data_sql/dispo_alim.csv')
# df_equilibre_prod = pd.read_csv('data_sql/equilibre_prod.csv')
# df_sous_nutrition = pd.read_csv('data_sql/sous_nutrition.csv')

# Connection à SQLite 

In [8]:
connection = sqlite3.connect("data_sql/db_projet_1.db")
cursor = connection.cursor()

## Création des 4 tables (vides)

In [10]:
try:
    cursor.execute('''
    CREATE TABLE population(
    pays TEXT NOT NULL,
    code_pays INT, 
    annee INT, 
    population REAL,
    PRIMARY KEY(code_pays, annee)
    )
    ''')
    print('TABLE population créé avec succès')
except sqlite3.Error as e:
    print("[Erreur] : ", e)

TABLE population créé avec succès


In [11]:
try :
    cursor.execute('''
    CREATE TABLE dispo_alim (
    pays TEXT NOT NULL, 
    code_pays INT, 
    annee INT, 
    produit TEXT, 
    origin TEXT, 
    dispo_alim_tonnes REAL, 
    dispo_alim_kcal_p_j REAL, 
    dispo_prot REAL, 
    dispo_mat_gr REAL,
    PRIMARY KEY(code_pays,annee, produit)
    )
    ''')
    print('TABLE dispo_alim créé avec succès')
except sqlite3.Error as e:
    print("[Erreur] : ", e)
    

TABLE dispo_alim créé avec succès


In [12]:
try : 
    cursor.execute('''
    CREATE TABLE equilibre_prod  (
    pays TEXT, 
    code_pays INT, 
    annee INT, 
    produit TEXT, 
    dispo_int REAL,
    alim_ani REAL, 
    semences REAL, 
    pertes REAL, 
    transfo REAL, 
    nourriture REAL, 
    autres_utilisations REAL,
    PRIMARY KEY(code_pays,annee, produit)
    )
    ''')
    print('TABLE equilibre_prod créé avec succès')     
except sqlite3.Error as e:
    print("[Erreur] : ", e)

TABLE equilibre_prod créé avec succès


In [13]:
try:
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sous_nutrition (
    pays TEXT, 
    code_pays INT, 
    annee INT, 
    nb_personnes REAL,
    PRIMARY KEY(code_pays,annee)
    )
    ''')
    print('TABLE sous_nutrition créé avec succès')
    
except sqlite3.Error as e:
    print("[Erreur] : ", e)

TABLE sous_nutrition créé avec succès


## Rajouter les données à chaque table (à partir des df) 

In [14]:
try:
    df_population.to_sql('population', con=connection, if_exists='append',index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [15]:
try :
    df_dispo_alim.to_sql('dispo_alim', con=connection, if_exists='append',index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [16]:
try :
    df_equilibre_prod.to_sql('equilibre_prod', con=connection, if_exists='append',index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [17]:
try:
    df_sous_nutrition.to_sql('sous_nutrition', con=connection, if_exists='append', index=False)
    connection.commit()
except sqlite3.Error as e:
    print("[Erreur] : ", e)

In [18]:
# Fermeture de la connextion avec SQLite
connection.close()